# 1. Import the necessary libraries for you to be able to query 4square API

In [93]:
import requests
import json
import pandas as pd
import os
from dotenv import load_dotenv
import pymongo

# 2. Get names and location of the 10 coffee places by passing Ironhack as your location

In [17]:
load_dotenv()

True

In [24]:
token = os.getenv("token")

In [20]:
lat_ih = 41.3977461
lon_ih = 2.187674417

In [21]:
def requests_for_foursquare (query, lat, lon, radius=500, limit=1):
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}"
    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("Request lost in the interweb")

In [22]:
requests_for_foursquare ("coffee", lat_ih, lon_ih, 500, 10)

{'results': [{'fsq_id': '530c5e6911d2be0b8d22e27a',
   'categories': [{'id': 13034,
     'name': 'Café',
     'short_name': 'Café',
     'plural_name': 'Cafés',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
      'suffix': '.png'}},
    {'id': 13035,
     'name': 'Coffee Shop',
     'short_name': 'Coffee Shop',
     'plural_name': 'Coffee Shops',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
      'suffix': '.png'}},
    {'id': 13065,
     'name': 'Restaurant',
     'short_name': 'Restaurant',
     'plural_name': 'Restaurants',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
      'suffix': '.png'}}],
   'chains': [],
   'closed_bucket': 'VeryLikelyOpen',
   'distance': 320,
   'geocodes': {'main': {'latitude': 41.396937, 'longitude': 2.191519},
    'roof': {'latitude': 41.396937, 'longitude': 2.191519}},
   'link': '/v3/places/530c5e6911d2be0b8d22e27a',
   'location': {'address': 'Calle

# 3. Now get the 5 CLOSEST coffe places. Make use of params: limit, near?

In [29]:
# Redefining the function so that it includes these new params:
def requests_for_foursquare (query, lat, lon, radius=500, limit=1, sort_by= "DISTANCE"):
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&sort={sort_by}&limit={limit}"
    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("Request lost in the interweb")

In [67]:
requests_for_foursquare("coffe", lat_ih, lon_ih, limit = 5)

{'results': [{'fsq_id': '5af43ea68a6f17002c872f81',
   'categories': [{'id': 13065,
     'name': 'Restaurant',
     'short_name': 'Restaurant',
     'plural_name': 'Restaurants',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
      'suffix': '.png'}}],
   'chains': [],
   'closed_bucket': 'VeryLikelyOpen',
   'distance': 72,
   'geocodes': {'drop_off': {'latitude': 41.397433, 'longitude': 2.188275},
    'main': {'latitude': 41.397229, 'longitude': 2.188428},
    'roof': {'latitude': 41.397229, 'longitude': 2.188428}},
   'link': '/v3/places/5af43ea68a6f17002c872f81',
   'location': {'address': 'Sancho de Avila, 22',
    'admin_region': 'Cataluña',
    'country': 'ES',
    'cross_street': 'Zamora',
    'formatted_address': 'Sancho de Avila, 22 (Zamora), 08018 Barcelona Catalunya',
    'locality': 'Barcelona',
    'postcode': '08018',
    'region': 'Catalunya'},
   'name': 'Forastera',
   'related_places': {},
   'timezone': 'Europe/Madrid'},
  {'fsq_id':

# 4. Define a function that returns name, lat, long for starbucks OR restaurants AND for any given location

In [64]:
def foursquare_places (venue, coordinates):
    '''This function returns the name and coordinates of the venue closest to the coordinates introduced.'''
    lat_query = coordinates.split(",")[0].strip()
    lon_query = coordinates.split(",")[1].strip()
    
    res = requests_for_foursquare(venue, lat_query, lon_query, 1000, 1)
    name = res["results"][0]["name"]
    lat = res["results"][0]["geocodes"]["main"]["latitude"]
    long = res["results"][0]["geocodes"]["main"]["longitude"]
    return name, lat, long

In [65]:
name, lat, long = foursquare_places ("restaurants", "41.3977461, 2.187674417")

In [66]:
print(name, lat, long)

L'Estoneta 41.399145 2.188496


# 5. Export it into data folder as a json: `data/coffe_shops.json`

In [95]:
dict_request = {
    "name":name,
    "lat":lat,
    "long": long
}
df = pd.DataFrame([dict_request])
df.to_json("data/coffe_shops.json")

# BONUS: load it into MongoDB in a collection named "ironcoffee" using pymongo

In [96]:
client = pymongo.MongoClient("localhost:27017")
db = client["Ironhack"]
c = db.get_collection("ironcoffee")
for index, row in df.iterrows():
    c.insert_one(dict(row))